In [1]:
import tensorflow as tf
import matrix_decompositions_tf as fctr

In [2]:
import transforms as transf
sigSz = (43,43)
noc = 3
nof = 4
fltrSz = (11,11)
batchSz = 5

In [3]:
y = tf.random.uniform(shape = (batchSz,sigSz[0],sigSz[1],nof,1),minval=0,maxval=1.0,dtype = 'float64')
fft = transf.fft2d_inner(fftSz=sigSz,dtype = 'float64')
ifft = transf.ifft2d_inner(fftSz=sigSz,dtype='complex128')
D = tf.random.uniform(shape=(fltrSz[0],fltrSz[1],noc,nof),minval=0.,maxval=1.0,dtype = 'float64')
divide_by_R = fctr.Coef_Divide_By_R(tf.expand_dims(D,axis=0),noc = D.shape[2])
D = tf.reshape(divide_by_R.D,divide_by_R.D.shape[1:])
Df = fft(tf.expand_dims(D,axis = 0))
Dy = ifft(tf.matmul(Df,fft(y)))

In [4]:
muld_sp = fctr.MulD_Sp(divide_by_R,dtype = 'float64')
print(y.shape)
Dy_sp = muld_sp(y)
print(Dy_sp.shape)

(5, 43, 43, 4, 1)
(5, 43, 43, 3, 1)


In [5]:
print(tf.reduce_max(tf.math.abs(Dy - Dy_sp)))

tf.Tensor(1.5916157281026244e-12, shape=(), dtype=float64)


In [6]:
print(D.shape)
print(y.shape)


(11, 11, 3, 4)
(5, 43, 43, 4, 1)


In [7]:
Dflipped = tf.reverse(D,axis=(0,1))
#ypad1 = tf.concat([y[slice(None),slice(-tf.cast((fltrSz[0] - 1)/2,'int32') - 1,None)],y,y[slice(None),slice(tf.cast((fltrSz[0] - 1)/2 - 1,'int32'),0,-1)]],axis = 1)
ypad1 = tf.concat([y[slice(None),slice(-(fltrSz[0] - 1),None)],y],axis = 1)

#ypad2 = tf.concat([ypad1[slice(None),slice(None),slice(-tf.cast((fltrSz[1] - 1)/2 ,'int32')- 1,None)],ypad1,ypad1[slice(None),slice(None),slice(tf.cast((fltrSz[1] - 1)/2 - 1,'int32'),0,-1)]],axis = 2)
ypad2 = tf.concat([ypad1[slice(None),slice(None),slice(-(fltrSz[1] - 1),None)],ypad1],axis = 2)
ypad2 = tf.reshape(ypad2,ypad2.shape[:-1])

print('ypad2_shape: ',ypad2.shape)
print('Dflipped_shape: ',Dflipped.shape)
outputs = tf.nn.conv2d(input = ypad2,filters = tf.transpose(Dflipped,perm=(0,1,3,2)),strides = 1,padding='VALID',dilations=1)
Dy_sp = tf.expand_dims(outputs,axis = -1)

ypad2_shape:  (5, 53, 53, 4)
Dflipped_shape:  (11, 11, 3, 4)


In [8]:
#print(Dtx.shape)
#print(Dtx_sp.shape)

In [9]:
print(Dy.shape)
print(Dy_sp.shape)

#print(tf.reduce_max(tf.math.abs(Dy[slice(None),slice(fltrSz[0] - 1,None),slice(fltrSz[1] -1,None)] - Dy_sp[slice(None),slice(1,-(fltrSz[0] - 2)),slice(1,-(fltrSz[1] - 2))])))
#Dy_shift = tf.roll(tf.roll(Dy,shift = -1,axis = 1),shift = -1,axis = 2)
print(tf.reduce_max(tf.math.abs(Dy - Dy_sp)))

(5, 43, 43, 3, 1)
(5, 43, 43, 3, 1)
tf.Tensor(1.5916157281026244e-12, shape=(), dtype=float64)


In [10]:
x = tf.random.uniform(shape = (batchSz,sigSz[0],sigSz[1],noc,1),minval=0.,maxval=1.0,dtype='float64')
Dtx = ifft(tf.matmul(Df,fft(x),adjoint_a = True))


In [11]:
Dflipped = tf.reverse(D,axis=(0,1))
xpad1 = tf.concat([x,x[slice(None),slice(0,(fltrSz[0] - 1))]],axis = 1)
xpad2 = tf.concat([xpad1,xpad1[slice(None),slice(None),slice(fltrSz[1] - 1)]],axis = 2)

outputs = tf.nn.conv2d_transpose(input = tf.reshape(xpad2,xpad2.shape[:-1]),filters = tf.transpose(Dflipped,perm=(0,1,3,2)),output_shape = (batchSz,sigSz[0] + fltrSz[0] - 1,sigSz[1] + fltrSz[0] - 1,nof),strides = 1,padding='SAME',dilations=1)
Dtx_sp = tf.expand_dims(outputs,axis=-1)
#Dtx_sp = Dtx_sp[slice(None),slice(3,-3),slice(3,-3)]

In [12]:
print(Dtx_sp.shape)
print(Dtx.shape)
temp_int = [tf.cast((fltrSz[ii] - 1)/2,'int32') for ii in range(2)]
print(tf.reduce_max(tf.math.abs(Dtx_sp[slice(None),slice(temp_int[0],-temp_int[0]),slice(temp_int[1],-temp_int[1])] - Dtx)))

(5, 53, 53, 4, 1)
(5, 43, 43, 4, 1)
tf.Tensor(1.1795009413617663e-12, shape=(), dtype=float64)


In [13]:
print(tf.reshape(Dtx[slice(0,1),slice(-6,None),slice(-6,None),slice(0,1)],shape = (6,6)))

tf.Tensor(
[[91.32629484 89.24625988 87.62192315 87.77430716 89.85091798 84.47329084]
 [85.08813723 87.02146019 85.99388984 88.53570207 85.6072492  85.44089484]
 [89.07810908 88.84574621 85.12278335 85.16859639 89.5360646  82.51883406]
 [87.4974313  90.25091964 83.55356735 85.56073482 89.41528729 88.56017958]
 [86.60785413 86.81892368 83.27389834 85.36236757 84.39084045 85.83779845]
 [89.53126316 89.45001263 87.84361076 84.12851208 83.48652541 83.30112097]], shape=(6, 6), dtype=float64)


In [14]:
print(tf.reshape(Dtx_sp[slice(0,1),slice(-9,-3),slice(-9,-3),slice(0,1)],shape = (6,6)))

tf.Tensor(
[[85.12278335 85.16859639 89.5360646  82.51883406 80.64763712 68.90605887]
 [83.55356735 85.56073482 89.41528729 88.56017958 80.53263609 73.85650604]
 [83.27389834 85.36236757 84.39084045 85.83779845 80.00647167 68.30240413]
 [87.84361076 84.12851208 83.48652541 83.30112097 79.65434817 70.89426138]
 [78.94676207 76.92079425 77.53451357 76.83131386 70.84114637 66.65242369]
 [68.84864781 72.72888565 69.09069986 71.445119   65.53894062 58.77933589]], shape=(6, 6), dtype=float64)


In [15]:
#print(tf.reduce_max(tf.math.abs(Dtx - Dtx_sp)))

Dy is a shifted version of Dy_sp.

In [16]:
#print(tf.reshape(Dy[slice(0,1),slice(None),slice(None),slice(0,1)],shape = (6,6)))

In [17]:
#print(tf.reshape(Dy_sp[slice(0,1),slice(None),slice(None),slice(0,1)],shape = (6,6)))

In [18]:
import matrix_decompositions_tf as fctr

print(Df.shape)

(1, 43, 22, 3, 4)


In [19]:
divide_by_R = fctr.Coef_Divide_By_R(tf.expand_dims(D,axis=0),noc = D.shape[2])

In [20]:
dhmul = fctr.DhMul(fft(divide_by_R.D),dtype = tf.complex128)

In [21]:
dmul = fctr.DMul(dhmul,dtype = tf.complex128)

In [22]:
print(x.shape)

(5, 43, 43, 3, 1)


In [23]:
Dhx = ifft(dhmul(fft(x)))

In [24]:
Dy = ifft(dmul(fft(y)))

In [25]:
print(Dhx.shape)

(5, 43, 43, 4, 1)


In [26]:
print(Dy.shape)

(5, 43, 43, 3, 1)


In [27]:
Dconv = fctr.MulD_Sp(divide_by_R,dtype = 'float64')

In [28]:
Dtconv = fctr.MulDT_Sp(divide_by_R,dtype = 'float64')

In [29]:
print(x.shape)

(5, 43, 43, 3, 1)


In [30]:
Dhx_sp = Dtconv(x)

In [31]:
Dy_sp = Dconv(y)

In [32]:
print(tf.reduce_max(tf.math.abs(Dhx_sp - Dhx)))

tf.Tensor(1.1795009413617663e-12, shape=(), dtype=float64)
